In [1]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import keras

import matplotlib.pyplot as plt
import math as mt

from keras.models import Sequential
from keras.layers import Dense , Dropout , Flatten
from keras.layers import MaxPooling2D
from keras.layers.convolutional import Conv2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils.np_utils import to_categorical
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras import backend as K
K.set_image_dim_ordering('th')

# Root directory of the project
ROOT_DIR = os.getcwd()
DEEPFASHION_DATA_DIR = os.path.join(ROOT_DIR, "fashion_data")


/Users/luke/.pyenv/versions/3.6.3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/luke/.pyenv/versions/3.6.3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# Load Originnal Data
Deepfashion dataset에서 category, imagepath, bbox 데이타를 load한다.

In [40]:
# category
val_category = np.genfromtxt(os.path.join(DEEPFASHION_DATA_DIR, "Anno", "list_category_img.txt"))
# string과 number가 섞여 있기 때문에 첫번째 인자가 non으로 오므로 삭제한다.
val_category = np.delete(val_category, 0, 1)
# 2차 Array를 1차 Array로 만들어준다.
val_category = val_category.flatten()


# bbox
val_bbox = np.genfromtxt(os.path.join(DEEPFASHION_DATA_DIR, "Anno", "list_bbox.txt"))
# string과 number가 섞여 있기 때문에 첫번째 인자가 non으로 오므로 삭제한다.
val_bbox = np.delete(val_bbox, 0, 1)
# 2차 Array를 1차 Array로 만들어준다.
# val_bbox = val_bbox.flatten()

# image path
val_imagepath = []
with open(os.path.join(DEEPFASHION_DATA_DIR, "Anno", "list_category_img.txt")) as list_category_img:
    for row in list_category_img:
        imagepath=row.strip()[:-1].strip().replace(' ', '_')
        val_imagepath.append(imagepath)
        
print(val_imagepath[0], val_category[0], val_bbox[0])

img/Sheer_Pleated-Front_Blouse/img_00000001.jpg 3.0 [ 72.  79. 232. 273.]


# Make Dataset for ML
Load한 data들의 Array를 하나의 Array로 합친다.
shuffle한다.

training set과 검증 셋으로 나눈다.
x_train, y_train, x_val, y_val 으로 저장한다.